In [ ]:
# This decorator can be used to implement strict FUNCTION OVERLOADING (not supported in Python due to duck-typing principle)
# Strict function overloading: the number of arguments is the same, the difference is in their types

In [1]:
from functools import singledispatch    # biến đổi một hàm tành một single-dispatched generic function

In [ ]:
@singledispatch       # dispatch xảy ra dựa vào kiểu của đối số thứ nhất 
def func(arg, verbose: bool = False) -> None:
    """Define a generic function.
    NOTE: Nếu kiểu dữ liệu của đối số đầu tiên khi gọi hàm chưa có hàm overloaded thì mặc định hàm generic sẽ được gọi.
    """
    if verbose:
        print(f"A single-dispatched generic function on first argument.", end=" ")
    print(arg)

# Add overloaded implementations to the above function: sử dụng thuộc tính register() của generic function
@func.register
def _(arg: int, verbose: bool = False) -> None:
    if verbose:
        print(f"Overloaded function for type int.", end=" ")
    print(arg)

@func.register
def _(arg: list, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for type list.", end=" ")
    print(*arg, sep=", ")

# When called, the generic function dispatches on the type of the first argument
func("Hello world", verbose=True)    # kiểu dữ liệu str chưa có hàm overloaded => mặc định sẽ gọi hàm generic
func(42, verbose=True)
func([1, 2, 3, 4], verbose=True)

A single-dispatched generic function on first argument. Hello world
Overloaded function for type int. 42
Overloaded function for type list. 1, 2, 3, 4


In [4]:
# Có thể gộp nhiều kiểu dữ liệu vào một hàm overloaded. 
@func.register
def _(arg: int | float, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for type int or float.", end=" ")
    print(arg)

from typing import Union
@func.register
def _(arg: Union[list, set], verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for type list or set.", end=" ")
    print(*arg, sep=", ")

func(42, verbose=True)
func(12.0, verbose=True)
func([4, 5, 6], verbose=True)
func({4, 5, 6}, verbose=True)

Overloaded function for type int or float. 42
Overloaded function for type int or float. 12.0
Overloaded function for type list or set. 4, 5, 6
Overloaded function for type list or set. 4, 5, 6


In [ ]:
# Ta có thể truyền kiểu dữ liệu vào thẳng thuộc tính register() thay vì sử dụng type hint
@func.register(complex)    # overloaded function cho kiểu dữ liệu complex
def _(arg, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for complex data type.", end=" ")
    print(arg.real, arg.imag)

# Với đối số là kiểu collection (list, tuple,...) và ta muốn typehint phần tử của collection đó (vi dụ: list[int]):
# truyền dispatch type vào register() decorator + truyền typehint vào function definition
@func.register(list)
def _(arg: list[int], verbose: bool = False):     # typehint chỉ có tác dụng cho static type checker, không có tác dụng at runtime 
    if verbose:
        print("Overloaded function for a list of integers.", end=" ")
    print(*arg, sep=", ")
    
func(2+3j, verbose=True)
func([1, 5, 7], verbose=True)
func(["foo", "bar", "baz"], verbose=True)     # vẫn gọi hàm dispatch với kiểu list[int] ở trên vì typehint chỉ dành cho static type cheker thôi 

Overloaded function for complex data type. 2.0 3.0
Overloaded function for a list of integers. 1, 5, 7
Overloaded function for a list of integers. foo, bar, baz


In [10]:
# Register các hàm đã được định nghĩa trước đó / hàm lambda: sử dụng register() ở dạng hàm
def nothing(arg, verbose: bool = False) -> None:
    """A pre-existing function.
    """
    if verbose:
        print("Overloaded function for None type.", end=" ")
    print("Nothing")

func.register(type(None), nothing)   # overload hàm nothing với kiểu None

func(None, verbose=True)

Overloaded function for None type. Nothing


In [ ]:
# @register() decorator trả về undecorated function, tức nó không làm thay đổi / wrap hàm ban đầu được register. 
# Nó chỉ đơn giản register hàm đó làm handler cho kiểu dữ liệu được chỉ định và trả về hàm ban đầu (undecorated)

# Điều này cũng dễ hiểu, tại vì có thể có nhiều hơn một hàm overloaded, mà chỉ có một hàm generic thì ghi đè làm sao được nhiều hàm vào
# một hàm generic duy nhất.

@singledispatch
def process(value):
    return f"Default handler for {value}."

# Using @register to add a type-specific handler for int
@process.register(int)
def _(value):
    return f"Processing an integer: {value}."

# @register does not modify the registered function
assert process.registry[int] is _     # The registered function is the same as the original (undecorated) one 
print(_(10))
print(process(10))

# Ứng dụng: decorator stacking, pickling, unit test

Processing an integer: 10.
Processing an integer: 10.


In [ ]:
from decimal import Decimal

@func.register(float)
@func.register(Decimal)
def func_number(arg, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for float or Decimal.", end=" ")
    print(arg)

func_number is func       # False vì decorator không wrap hàm undecorated

False

In [21]:
# Overloaded with abc
from collections.abc import Mapping
@func.register
def _(arg: Mapping, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for the abc type Mapping.", end=" ")
    for key, value in arg.items():
        print(key, value, sep=": ")

func({"a": "b"}, verbose=True)

Overloaded function for the abc type Mapping. a: b


In [ ]:
# Kiểm tra overloaded function mà hàm generic sẽ chọn cho một kiểu dữ liệu cụ thể: sử dụng dispatch() attribute của generic function

from inspect import getsource

print(func.dispatch(float))              # trả về hàm được register cho kiểu float 
print(getsource(func.dispatch(float)))   # lấy source code của hàm register cho kiểu float

print(func.dispatch(dict)) 
print(getsource(func.dispatch(dict)))

<function func_number at 0x00000198C1D37F60>
@func.register(float)
@func.register(Decimal)
def func_number(arg, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for float or Decimal.", end=" ")
    print(arg)

<function _ at 0x00000198C1D9CF40>
@func.register
def _(arg: Mapping, verbose: bool = False) -> None:
    if verbose:
        print("Overloaded function for the abc type Mapping.", end=" ")
    for key, value in arg.items():
        print(key, value, sep=": ")



In [29]:
# Access all registered implementations (overloaded functions), use read-only `registry` attribute
func.registry

mappingproxy({object: <function __main__.func(arg, verbose: bool = False) -> None>,
              int: <function __main__._(arg: int, verbose: bool = False) -> None>,
              list: <function __main__._(arg: list[int], verbose: bool = False)>,
              complex: <function __main__._(arg, verbose: bool = False) -> None>,
              NoneType: <function __main__.nothing(arg, verbose: bool = False) -> None>,
              decimal.Decimal: <function __main__.func_number(arg, verbose: bool = False) -> None>,
              float: <function __main__.func_number(arg, verbose: bool = False) -> None>,
              bool: <function __main__._(arg, verbose: bool = False) -> None>,
              collections.abc.Mapping: <function __main__._(arg: collections.abc.Mapping, verbose: bool = False) -> None>})

In [32]:
func.registry.keys()

dict_keys([<class 'object'>, <class 'int'>, <class 'list'>, <class 'complex'>, <class 'NoneType'>, <class 'decimal.Decimal'>, <class 'float'>, <class 'bool'>, <class 'collections.abc.Mapping'>])

In [33]:
func.registry.values()

dict_values([<function func at 0x00000198C14C5F80>, <function _ at 0x00000198C14C6020>, <function _ at 0x00000198C1CA8400>, <function _ at 0x00000198C1CA8040>, <function nothing at 0x00000198C14C7880>, <function func_number at 0x00000198C1D37F60>, <function func_number at 0x00000198C1D37F60>, <function _ at 0x00000198C1D9F100>, <function _ at 0x00000198C1D9CF40>])

In [ ]:
print(func.registry[float])
print(func.registry[int])
print(func.registry[list])
print(func.registry[object])      # generic function

<function func_number at 0x00000198C1D37F60>
<function _ at 0x00000198C14C6020>
<function _ at 0x00000198C1CA8400>
<function func at 0x00000198C14C5F80>
